# Source dump

- https://www.tensorflow.org/guide/eager
- http://web.archive.org/web/20200802155249/https://www.tensorflow.org/guide/eager
- https://github.com/tensorflow/docs/blob/3ce5b40191bf8ff8f3663b015815940fbc5907e5/site/en/guide/eager.ipynb

# Eager execution

TensorFlow's eager execution is an imperative programming environment that evaluates operation immediately, without building graphs: operations return concrete values instead of constructing a computational graph to run later. This makes it easy to get started with TensorFlow and debug models, and it reduces boilerplate as well. To follow along with this guide, run the code samples below in an interactive `python` interpreter.

Eager execution is a flexible machine learning platform for research and experimentation, providing:

- _An intuitive interface_ - Structure your code naturally and use Python data structures. Quickly iterate on small models and small data.
- _Easier debugging_ - Call ops directly to inspect running models and test changes. Use standard Python debugging tools for immediate error reporting.
- _Natural control flow_ - Use Python control flow instead of graph control flow, simplifying the specification of dynamic models.

Eager execution supports most TensorFlow operations and GPU acceleration.

> __Note:__ Some models may experience increased overhead with eager execution enabled. Performance improvements are ongoing, but please file a bug if you find a problem and share your benchmarks.

## Setup and basic usage

In [1]:
import os
import tensorflow as tf

In [2]:
import cProfile

In TensorFlow 2.0, eager execution is enabled by default.

In [3]:
tf.executing_eagerly()

True

Now you can run TensorFlow operations and the results will return immediately:

In [4]:
x = [[2.]]
m = tf.matmul(x, x)
print('hello,', m)

hello, tf.Tensor([[4.]], shape=(1, 1), dtype=float32)


Enabling eager execution changes how TensorFlow operations behave - now they immediately evaluate and return their values to Python. `tf.Tensor` objects reference concrete values instead of symbolic handles to nodes in a computational graph. Since there isn't a computational graph to build and run later in a session, it's easy to inspect results using `print()` or a debugger. Evaluating, printing, and checking tensor values does not break the flow for computing gradients.

Eager execution works nicely with NumPy. NumPy operations accept `tf.Tensor` arguments. The TensorFlow `tf.math` operations convert Python objects and NumPy arrays to `tf.Tensor` objects. The `tf.Tensor.numpy` method returns the object's value as a NumPy `ndarray`.

In [5]:
a = tf.constant([[1, 2], [3, 4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [6]:
# Broadcasting support
b = tf.add(a, 1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [7]:
# Operator overloading is supported
print(a * b)

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [8]:
# Use NumPy values
import numpy as np

c = np.multiply(a, b)
print(c)

[[ 2  6]
 [12 20]]


In [9]:
# Obtain numpy value from a tensor:
print(a.numpy())

[[1 2]
 [3 4]]


## Dynamic control flow

A major benefit of eager execution is that all the functionality of the host language is available while your model is executing. So, for example, it is easy to fizzbuzz:

In [10]:
def fizzbuzz(max_num):
    counter = tf.constant(0)
    max_num = tf.convert_to_tensor(max_num)
    for num in range(1, max_num.numpy()+1):
        num = tf.constant(num)
        if int(num % 3) == 0 and int(num % 5) == 0:
            print('FizzBuzz')
        elif int(num % 3) == 0:
            print('Fizz')
        elif int(num % 5) == 0:
            print('Buzz')
        else:
            print(num.numpy())
        counter += 1

In [11]:
fizzbuzz(15)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz


This has conditionals that depend on tensor values and it prints these values at runtime.

## Eager training

### Computing gradients

Automatic differentiation is useful for implementing machine learning algorithms such as backpropagration for training neural networks. During eager execution, use `tf.GradientTape` to trace operations for computing gradients later.

You can use `tf.GradientTape` to train and/or compute gradients in eager. It is especially useful for complicated training loops.

Since different operations can occur during each call, all forward-pass operations get recorded to a "tape". To compute the gradient, play the tape backwards and then discard. A particular `tf.GradientTape` can only compute one gradient; subsequent calls throw a runtime error.

In [12]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
    loss = w * w

grad = tape.gradient(loss, w)
print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


### Train a model

The following example creates a multi-layer model that classifies the standard MNIST handwritten digits. It demonstrates the optimizer and layer APIs to build trainable graphs in an eager execution environment.

In [14]:
import tensorflow.keras

In [1]:
from tensorflow import keras

In [2]:
keras

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "d:\python\python37\lib\site-packages\IPython\core\formatters.py", line 224, in catch_format_error
    r = method(self, *args, **kwargs)
  File "d:\python\python37\lib\site-packages\IPython\core\formatters.py", line 702, in __call__
    printer.pretty(obj)
  File "d:\python\python37\lib\site-packages\IPython\lib\pretty.py", line 394, in pretty
    return _repr_pprint(obj, self, cycle)
  File "d:\python\python37\lib\site-packages\IPython\lib\pretty.py", line 684, in _repr_pprint
    output = repr(obj)
  File "<frozen importlib._bootstrap>", line 295, in _module_repr
  File "d:\python\python37\lib\site-packages\tensorflow\__init__.py", line 50, in __getattr__
    module = self._load()
  File "d:\python\python37\lib\site-packages\tensorflow\__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "d:\python\python37\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[leve

ModuleNotFoundError: No module named 'tensorflow_core.keras'

In [19]:
# Fetch and format the mnist data
(mnist_images, mnist_labels), _ = keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices((tf.cast(mnist_images[..., tf.newaxis]/255, tf.float32), tf.cast(mnist_labels, tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "d:\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-3d458f16a66c>", line 2, in <module>
    (mnist_images, mnist_labels), _ = keras.datasets.mnist.load_data()
  File "d:\python\python37\lib\site-packages\tensorflow\__init__.py", line 50, in __getattr__
    module = self._load()
  File "d:\python\python37\lib\site-packages\tensorflow\__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "d:\python\python37\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1006, in _gcd_import
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 965, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'tensorflow_core.ker

ModuleNotFoundError: No module named 'tensorflow_core.keras'